# Worksheet 05

Name: Stone Harris
UID: U41533031

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

1. Initialize the centroids [0, 2].
2. Assign each data point to the nearest centroid: Points closer to 0: [0, 0.5, 1.5] ; Points closer to 2: [2, 6, 6.5, 7]
3. Recompute centroids: New 1: Average of [0, 0.5, 1.5] = (0+0.5+1.5)/3 = 0.67 New 2: Average of [2, 6, 6.5, 7] = (2+6+6.5+7)/4 = 5.38
4. Repeat: New points closer to 0.67: [0, 0.5, 1.5, 2] New points closer to 5.38: [6, 6.5, 7]
5. Recompute centroids again: New 1: Average of [0, 0.5, 1.5, 2] = 1 ; New 2: Average of [6, 6.5, 7] = 6.5
6. Could keep going, just repeating

b) Describe in plain english what the cost function for k means is.

The k-means cost function is basically how we score how good our clustering is. It adds up all the distances from each point to the center of its cluster; we want this score to be as low as possible because it means all the points in a cluster are bunched up close to the center, not scattered all over the place.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Different solutions to the K-means algorithm can occur because it often starts with random cluster centers. So based on where these initial centers are placed, the algorithm might group the data points differently each time it's run, leading to varied clustering results for the same dataset.

d) Does Lloyd's Algorithm always converge? Why / why not?

Lloyd's algorithm usually converges to a solution because it iteratively optimizes the cluster assignments and centroids, but it might not find the optimal clustering due to local minima. It doesn't necessarily guarantee the best solution though.

e) Follow along in class the implementation of Kmeans

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
    
    def is_unassigned(self):
        return self.assignment[1] == -1
        
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]

    def dist(self, x, y):
        return sum((x - y)**2) ** (1/2)
    
    
    def initialize(self):
        return self.data[np.random.choice(len(self.data), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
       for i in range(len(c1)):
           if c1[i] in c2:
                return True
       return False
    
    def assign(self):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if new_dist < temp_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist

    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j))
        return
    
    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snaps.append(self.snap(centers))
        
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
        
        return
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)